In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:98% !important; }</style>"))

from sklearn.externals.joblib import Parallel, delayed

import pandas as pd
import numpy as np
import scipy as sp

import datetime
from collections import Counter
from sklearn.externals.joblib import Parallel, delayed
from IPython.display import display
from scipy import stats

import copy
from itertools import tee, izip

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def DiagCodeType(x):
    if(pd.isnull(x)):
        return(0)
    if(not unicode(str(x), 'utf-8').isnumeric()):
        return 1
    x = int(str(x[0:3]))
    if((x>=0)&(x<=139)):
        return 2
    if((x>=140)&(x<=239)):
        return 3
    if((x>=240)&(x<=279)):
        return 4
    if((x>=280)&(x<=289)):
        return 5
    if((x>=290)&(x<=319)):
        return 6
    if((x>=320)&(x<=389)):
        return 7
    if((x>=390)&(x<=459)):
        return 8
    if((x>=460)&(x<=519)):
        return 9
    if((x>=520)&(x<=579)):
        return 10
    if((x>=580)&(x<=629)):
        return 11
    if((x>=630)&(x<=679)):
        return 12
    if((x>=680)&(x<=709)):
        return 13
    if((x>=710)&(x<=739)):
        return 14
    if((x>=740)&(x<=759)):
        return 15
    if((x>=760)&(x<=779)):
        return 16
    if((x>=780)&(x<=799)):
        return 17
    if((x>=800)&(x<=999)):
        return 18
    return 0

In [3]:
def BuildFishersTest(col,week_vectors,weekend_vectors,data_label):
    week_values = week_vectors[col].values
    weekend_values = weekend_vectors[col].values
    a = weekend_vectors[col].value_counts().get(1,0)
    b = week_vectors[col].value_counts().get(1,0)
    c = weekend_vectors[col].value_counts().get(0,0)
    d = week_vectors[col].value_counts().get(0,0)
    try:
        col_label = data_label[data_label['ITEMID']==col]
        if(col_label.shape[0]==0):
            col_label = str(col)
        else:
            col_label = col_label['LABEL'].iloc[0]
    except:
        col_label = str(col)
    try:
        t = stats.fisher_exact([[a,b],[c,d]])
    except:
        return (col_label,np.nan,np.nan,np.nan,np.nan)

    return(col_label,np.mean(week_values),np.mean(weekend_values),t[0],t[1])

def BuildResultsVector(a,b,data_label,col):
    try:
        col_label = data_label[data_label['ITEMID']==int(col)]
        #col_label = pd.DataFrame()
        if(col_label.shape[0]==0):
            col_label = str(col)
        else:
            col_label = col_label['LABEL'].iloc[0]
    except:
        col_label = str(col)
    if((not a) | (not b)):
        return (col_label,np.nan,np.nan,np.nan,np.nan)
    try:
        t = stats.mannwhitneyu(a,b)
    except:
        return (col_label,np.nan,np.nan,np.nan,np.nan)

    return(col_label,np.mean(a),np.mean(b),t[0],t[1])
    

def TimeUntilFirstColumn(col,table,time_col,item_col,data_label):
    a = expired_pat_week.apply(lambda x: TimeUntilFirstAction(x['hadm_id'],x['ICU_Time'],data_week,time_col,specific_value=(item_col,col)),axis=1).values
    a = [x for x in a if pd.notnull(x)]

    b = expired_pat_weekend.apply(lambda x: TimeUntilFirstAction(x['hadm_id'],x['ICU_Time'],data_weekend,time_col,specific_value=(item_col,col)),axis=1).values
    b = [x for x in b if pd.notnull(x)]
    return(BuildResultsVector(a,b,data_label,col))

def TimeBetweenColumn(col,table,time_col,item_col,data_label):
    a = expired_pat_week.apply(lambda x: TimeBetweenItems(x['hadm_id'],x['ICU_Time'],data_week,time_col,specific_value=(item_col,col)),axis=1).values
    a = [x for x in a if pd.notnull(x)]

    b = expired_pat_weekend.apply(lambda x: TimeBetweenItems(x['hadm_id'],x['ICU_Time'],data_weekend,time_col,specific_value=(item_col,col)),axis=1).values
    b = [x for x in b if pd.notnull(x)]

    return(BuildResultsVector(a,b,data_label,col))

def TotalItemColumn(col,table,time_col,item_col,data_label):
    a = expired_pat_week.apply(lambda x: TotalItem(x['hadm_id'],x['ICU_Time'],data_week,time_col,(item_col,col)),axis=1).values
    b = expired_pat_weekend.apply(lambda x: TotalItem(x['hadm_id'],x['ICU_Time'],data_weekend,time_col,(item_col,col)),axis=1).values
    return(BuildResultsVector(list(a),list(b),data_label,col))

def TotalAmountColumn(col,table,amount_col,item_col,data_label):
    a = expired_pat_week.apply(lambda x: TotalAmountItem(x['hadm_id'],x['ICU_Time'],data_week,amount_col,(item_col,col)),axis=1)
    b = expired_pat_weekend.apply(lambda x: TotalAmountItem(x['hadm_id'],x['ICU_Time'],data_weekend,amount_col,(item_col,col)),axis=1)
    concat_mean = BuildResultsVector(list(a[0].values),list(b[0].values),data_label,col)
    concat_max = BuildResultsVector(list(a[1].values),list(b[1].values),data_label,col)
    concat_min = BuildResultsVector(list(a[2].values),list(b[2].values),data_label,col)
    concat_std = BuildResultsVector(list(a[3].values),list(b[3].values),data_label,col)
    longColumn = list(concat_mean)
    longColumn.extend(list(concat_max))
    longColumn.extend(list(concat_min))
    longColumn.extend(list(concat_std))
    return(longColumn)

def pairwise(iterable):
    a, b = tee(iterable)
    next(b, None)
    return izip(a, b)

def TimeBetweenItems(hadm_id,icu_time,table,time_column,specific_value=None):
    subset = copy.deepcopy(table[table['HADM_ID']==hadm_id])
    subset.dropna(subset=[time_column],inplace=True)
    if(specific_value!=None):
        subset = subset[subset[specific_value[0]]==specific_value[1]]
    time_diff_list = []
    for (i1, row1), (i2, row2) in pairwise(subset.sort_values(time_column).iterrows()):
        time_diff = float((row2[time_column]-row1[time_column]).total_seconds())/60/60
        if(time_diff>0):
            time_diff_list.append(time_diff)
    if(time_diff_list==[]):
        return np.nan
    return(np.mean(time_diff_list))

def TimeUntilFirstAction(hadm_id,icu_time,table,time_column,specific_value=None):
    subset = copy.deepcopy(table[table['HADM_ID']==hadm_id])
    subset.dropna(subset=[time_column],inplace=True)
    if(specific_value!=None):
        subset = subset[subset[specific_value[0]]==specific_value[1]]
    if(subset.shape[0]==0):
        return np.nan
    first_item = subset.sort_values(time_column).head(1)[time_column].iloc[0]
    time_diff = first_item-icu_time
    return(float(time_diff.total_seconds())/60/60)

def TotalItem(hadm_id,icu_time,table,time_column,specific_value):
    subset = copy.deepcopy(table[table['HADM_ID']==hadm_id])
    subset = subset[subset[specific_value[0]]==specific_value[1]]
    return(subset.shape[0])

def TotalAmountItem(hadm_id,icu_time,table,amount_col,specific_value):
    subset = copy.deepcopy(table[table['HADM_ID']==hadm_id])
    subset = subset[subset[specific_value[0]]==specific_value[1]]
    subset.dropna(subset=[amount_col],inplace=True)
    if(subset.shape[0]==0):
        return pd.Series([np.nan,np.nan,np.nan,np.nan])
    temp = subset[amount_col].values
    temp = [float(x) for x in temp if unicode(str(x), 'utf-8').isnumeric()]   
    if(len(temp)==0):
        return pd.Series([np.nan,np.nan,np.nan,np.nan])
    return pd.Series([np.nanmean(temp), np.max(temp), np.min(temp), np.std(temp)])

In [4]:
def ReturnNHours(hadm_id,icu_time,table,column,time_window):
    subset = copy.deepcopy(table[table['HADM_ID']==hadm_id])
    return(subset[(pd.notnull(subset[column])) & (subset[column]>=(icu_time)) & (subset[column]<=(icu_time+datetime.timedelta(hours=time_window)))])

def MakeData(table,column,time_window,expired_pat_weekend,expired_pat_week):
    data_weekend = pd.DataFrame()
    for row in expired_pat_weekend.iterrows():
        data_weekend = pd.concat([data_weekend,ReturnNHours(row[1]['hadm_id'],row[1]['ICU_Time'],table,column,time_window)])
        
    data_week = pd.DataFrame()
    for row in expired_pat_week.iterrows():
        data_week = pd.concat([data_week,ReturnNHours(row[1]['hadm_id'],row[1]['ICU_Time'],table,column,time_window)])
    return((data_weekend,data_week))

def LoadMIMICTables(table_name,time_columns,time_formats):
    table = pd.read_csv('../MIMICIII/'+table_name+'.csv')
    #Clean Data
    table = table[table['HADM_ID'].isin(expired_pat_id)]
    for index,item in enumerate(time_columns):
        table[item] = pd.to_datetime(table[item], format=time_formats[index])
    return(table)

def GetPossibleItems(table,column):
    #possible_items = table[column].unique()
    possible_items = [x[0] for x in Counter(table[column]).most_common(100)]
    possible_items = [x for x in possible_items if pd.notnull(x)]
    return(possible_items)

def MakeItemVectors(table,column,data_weekend,data_week):
    vector_mapping = {}
    vector_list = []
    possible_items = GetPossibleItems(table,column)
    possible_items_set = set(possible_items)
    for index,item in enumerate(possible_items):
        vector_list.append(item)
        vector_mapping[item] = index
    
    data_weekend_vectors = []
    for row in data_weekend['HADM_ID'].unique():
        temp_vector = np.zeros(len(possible_items))
        for row1 in table[table['HADM_ID']==row][column].unique():
            if((pd.isnull(row1)) | (row1 not in possible_items_set) ):
                continue
            temp_vector[vector_mapping[row1]] = 1
        data_weekend_vectors.append(list(temp_vector))
        
    data_week_vectors = []
    for row in data_week['HADM_ID'].unique():
        temp_vector = np.zeros(len(possible_items))
        for row1 in table[table['HADM_ID']==row][column].unique():
            if((pd.isnull(row1)) | (row1 not in possible_items_set) ):
                continue
            temp_vector[vector_mapping[row1]] = 1
        data_week_vectors.append(list(temp_vector))
    return((data_weekend_vectors,data_week_vectors,vector_list))

# Analysis Start Here

In [5]:
#Load Data
pat_data = pd.read_pickle('pat_data.pkl')

#Clean Data
#Cleaning
pat_data['Primary_Diag_Code'] = pat_data['Primary_Diag'].apply(lambda x: DiagCodeType(x))
pat_data = pat_data[pat_data.Admit_Type != 'NEWBORN']
pat_data = pat_data[pat_data.Admit_Unit != 'NICU']
pat_data = pat_data[pat_data.Lag_Time > 0]
pat_data = pat_data.replace(to_replace=['TRANSFER FROM SKILLED NUR','TRANSFER FROM OTHER HEALT','TRSF WITHIN THIS FACILITY','TRANSFER FROM HOSP/EXTRAM'], value='TRANSFER')
pat_data = pat_data[pat_data.Admit_LOC.isin(['EMERGENCY ROOM ADMIT','TRANSFER','CLINIC REFERRAL/PREMATURE','PHYS REFERRAL/NORMAL DELI'])]

#pat_data.loc[(pat_data.DOW == 5)|(pat_data.DOW == 6), 'weekend'] = 1
#pat_data.weekend = pat_data.weekend.fillna(0)

pat_data = pat_data[pat_data['Expire']==1]

#subset = pat_data[(pat_data['Admit_Unit']=='CSRU')  & (pat_data['Primary_Diag_Code']==12)]
subset = pat_data[(pat_data['Admit_LOC']=='PHYS REFERRAL/NORMAL DELI')]

#Controlled Data Test
expired_pat_weekend = subset[(subset['DOW'].isin([5,6]))]
expired_pat_week = subset[(subset['DOW'].isin([0,1,2,3,4]))]
expired_pat_id = pat_data['hadm_id'].values

In [6]:
d_items = pd.read_csv('../MIMICIII/D_ITEMS.csv')
d_lab_items = pd.read_csv('../MIMICIII/D_LABITEMS.csv')
#Check Datetime

In [7]:
subset.head()

,pat_id,hadm_id,Admit_Type,Admit_LOC,Insurance,Ethnicity,Expire,Admit_Time,Gender,Admit_Unit,...,NoICU,Transfer_Flag,Primary_Diag,DRG,Age,Hospital_LOS,Lag_Time,TOD,DOW,Primary_Diag_Code
127,12,112213,ELECTIVE,PHYS REFERRAL/NORMAL DELI,Medicare,WHITE,1,2104-08-07 10:15:00,M,SICU,...,1,0,1570,193.0,72.420348,12.695833,15.888056,Night,4,3
855,1224,168781,EMERGENCY,PHYS REFERRAL/NORMAL DELI,Medicare,UNKNOWN/NOT SPECIFIED,1,2187-06-11 19:44:00,F,CSRU,...,1,0,5109,75.0,74.821431,11.450694,92.552778,Day,4,9
1007,1427,178731,ELECTIVE,PHYS REFERRAL/NORMAL DELI,Medicare,WHITE,1,2161-12-08 07:15:00,F,SICU,...,1,0,44021,1694.0,77.910417,44.121528,16.188056,Night,1,8
1015,1436,178183,ELECTIVE,PHYS REFERRAL/NORMAL DELI,Private,WHITE,1,2156-04-20 10:00:00,M,SICU,...,1,1,1983,224.0,46.307991,16.083333,81.087500,Night,4,3
1038,1537,123829,ELECTIVE,PHYS REFERRAL/NORMAL DELI,Medicare,UNKNOWN/NOT SPECIFIED,1,2163-12-19 12:00:00,M,CSRU,...,1,0,4240,483.0,74.872603,32.231944,2.409167,Day,0,8


In [8]:
clinical_data = [('MICROBIOLOGYEVENTS','CHARTTIME','SPEC_ITEMID','NA','%Y-%m-%d %H:%M:%S'),('LABEVENTS','CHARTTIME','ITEMID','VALUENUM','%Y-%m-%d %H:%M:%S'),
                 ('PRESCRIPTIONS','STARTDATE','DRUG','DOSE_VAL_RX','%Y-%m-%d %H:%M:%S'),('PROCEDUREEVENTS_MV','STARTTIME','ITEMID','VALUE','%Y-%m-%d %H:%M:%S'),
                 ('CPTEVENTS','CHARTDATE','CPT_CD','NA','%Y-%m-%d %H:%M:%S'),('DATETIMEEVENTS','CHARTTIME','ITEMID','NA','%Y-%m-%d %H:%M:%S'),
                 ('NOTEEVENTS','CHARTDATE','CATEGORY','NA','%Y-%m-%d'),('CHARTEVENTS','CHARTTIME','ITEMID','VALUENUM','%Y-%m-%d %H:%M:%S')]

# clinical_data = [('MICROBIOLOGYEVENTS','CHARTTIME','SPEC_ITEMID','%Y-%m-%d %H:%M:%S'),('LABEVENTS','CHARTTIME','ITEMID','%Y-%m-%d %H:%M:%S')]

# clinical_data = [('MICROBIOLOGYEVENTS','CHARTTIME','SPEC_ITEMID','%Y-%m-%d %H:%M:%S'),('LABEVENTS','CHARTTIME','ITEMID','%Y-%m-%d %H:%M:%S'),
#                  ('PRESCRIPTIONS','STARTDATE','DRUG','%Y-%m-%d %H:%M:%S'),('PROCEDUREEVENTS_MV','STARTTIME','ITEMID','%Y-%m-%d %H:%M:%S'),
#                  ('CPTEVENTS','CHARTDATE','CPT_CD','%Y-%m-%d %H:%M:%S'),('DATETIMEEVENTS','CHARTTIME','ITEMID','%Y-%m-%d %H:%M:%S'),
#                  ('NOTEEVENTS','CHARTDATE','CATEGORY','%Y-%m-%d')]

clinical_data_tables = []
for table,time_col,item_col,amount_col,date_label in clinical_data:
    proc_table = LoadMIMICTables(table,[time_col],[date_label])
    if(table=='LABEVENTS'):
        clinical_data_tables.append((table,time_col,item_col,amount_col,proc_table,d_lab_items))
    else:
        clinical_data_tables.append((table,time_col,item_col,amount_col,proc_table,d_items))
        
input_cv = LoadMIMICTables('INPUTEVENTS_CV',['CHARTTIME'],['%Y-%m-%d %H:%M:%S'])
input_mv = LoadMIMICTables('INPUTEVENTS_MV',['STARTTIME'],['%Y-%m-%d %H:%M:%S'])
input_mv = input_mv.rename(index=str, columns={"STARTTIME": "CHARTTIME"})
input_columns = ['HADM_ID','CHARTTIME','ITEMID','AMOUNT','CGID']
input_data = pd.concat([input_cv[input_columns].reset_index(drop=True),input_mv[input_columns].reset_index(drop=True)])
clinical_data_tables.append(('INPUTITEMS','CHARTTIME','ITEMID','AMOUNT',input_data,d_items))

time_span = 24

/afs/crc.nd.edu/user/l/lfaust/anaconda3/envs/py27/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/afs/crc.nd.edu/user/l/lfaust/anaconda3/envs/py27/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (4,5,7,11) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/afs/crc.nd.edu/user/l/lfaust/anaconda3/envs/py27/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (13) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):
/afs/crc.nd.edu/user/l/lfaust/anaconda3/envs/py27/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(

In [ ]:
for table_label,time_col,item_col,amount_col,table,data_label in clinical_data_tables: 
    data_weekend,data_week = MakeData(table,time_col,time_span,expired_pat_weekend,expired_pat_week)
    vector_items = MakeItemVectors(table,item_col,data_weekend,data_week)
    weekend_vectors = pd.DataFrame(vector_items[0],columns=vector_items[2])
    week_vectors = pd.DataFrame(vector_items[1],columns=vector_items[2])
    
    class_weekend = list([1]*weekend_vectors.shape[0])
    class_week = list([0]*week_vectors.shape[0])
    classes = class_weekend+class_week
    print(week_vectors.shape)
    print(weekend_vectors.shape)

    data = []
    for col in vector_items[2]:
        data.append(BuildFishersTest(col,week_vectors,weekend_vectors,data_label))
        
    print(table_label)
    data = pd.DataFrame(data,columns=['Column','% Week','%Weekend','OR','P-Val'])
    data = data.dropna()
    display(data[data['P-Val']<0.05])   
    print(' ')

(123, 73)
(28, 73)
MICROBIOLOGYEVENTS


,Column,% Week,%Weekend,OR,P-Val
6,CATHETER TIP-IV,0.382114,0.678571,3.413712,0.005779
18,Immunology (CMV),0.113821,0.285714,3.114286,0.033837
28,Blood (EBV),0.048780,0.178571,4.239130,0.031815
30,BONE MARROW - CYTOGENETICS,0.056911,0.178571,3.602484,0.047343


 
(209, 100)
(47, 100)
LABEVENTS


,Column,% Week,%Weekend,OR,P-Val
46,Eosinophils,0.741627,0.893617,2.926452,0.033500
47,Lymphocytes,0.741627,0.893617,2.926452,0.033500
48,Monocytes,0.741627,0.893617,2.926452,0.033500
49,Neutrophils,0.741627,0.893617,2.926452,0.033500
50,Basophils,0.741627,0.893617,2.926452,0.033500
52,"Creatine Kinase, MB Isoenzyme",0.679426,0.872340,3.224178,0.007266
56,Bands,0.636364,0.829787,2.785714,0.010122
59,Troponin T,0.440191,0.659574,2.463995,0.009271
79,Atypical Lymphocytes,0.540670,0.723404,2.221920,0.023051
80,Metamyelocytes,0.540670,0.723404,2.221920,0.023051


 
(172, 100)
(38, 100)


...

,Column,% Week,%Weekend,OR,P-Val
10,Social Work,0.020101,0.085106,4.534884,0.045585


 
(208, 100)
(47, 100)
CHARTEVENTS


,Column,% Week,%Weekend,OR,P-Val


 
(204, 100)
(47, 100)
INPUTITEMS


,Column,% Week,%Weekend,OR,P-Val
27,Milrinone,0.093137,0.000000,0.000000,0.028856
43,Sandostatin,0.019608,0.085106,4.651163,0.042597
50,Labetolol,0.014706,0.085106,6.232558,0.024699


In [ ]:
#Time version
for table_label,time_col,item_col,amount_col,table,data_label in clinical_data_tables: 
    data_weekend,data_week = MakeData(table,time_col,time_span,expired_pat_weekend,expired_pat_week)
    possible_items = GetPossibleItems(table,item_col)

    data = Parallel(n_jobs=-1)(delayed(TimeUntilFirstColumn)(col,table,time_col,item_col,data_label) for col in possible_items)
    print("Time Until First Action")
    print(table_label)
    data = pd.DataFrame(data,columns=['Column','Mean Time Week','Mean Time Weekend','U-Stat','P-Val'])
    data = data.dropna()
    display(data[data['P-Val']<0.05])   
    print(' ')
    
    data = Parallel(n_jobs=-1)(delayed(TimeBetweenColumn)(col,table,time_col,item_col,data_label) for col in possible_items)
    print("Time Between")
    print(table_label)
    data = pd.DataFrame(data,columns=['Column','Mean Time Week','Mean Time Weekend','U-Stat','P-Val'])
    data = data.dropna()
    display(data[data['P-Val']<0.05])   
    print(' ')

Time Until First Action
MICROBIOLOGYEVENTS


,Column,Mean Time Week,Mean Time Weekend,U-Stat,P-Val


 
Time Between
MICROBIOLOGYEVENTS


,Column,Mean Time Week,Mean Time Weekend,U-Stat,P-Val


 
Time Until First Action
LABEVENTS


...

,Column,Mean Time Week,Mean Time Weekend,U-Stat,P-Val
2,Arterial line Insertion Date,4.505793,3.408810,63.0,0.046841
4,Arterial Line Dressing Change,4.369975,2.962143,45.0,0.020033
5,Arterial line Tubing Change,4.414665,2.962143,45.0,0.018187
13,INV#2 Cap Change [MM],3.671429,4.150000,45.0,0.036725


 
Time Until First Action
NOTEEVENTS


,Column,Mean Time Week,Mean Time Weekend,U-Stat,P-Val
7,Echo,8.483832,3.350035,63.0,0.010110
8,General,3.646746,9.966778,4.0,0.017378


 
Time Between
NOTEEVENTS


,Column,Mean Time Week,Mean Time Weekend,U-Stat,P-Val


In [ ]:
#Total Items
for table_label,time_col,item_col,amount_col,table,data_label in clinical_data_tables:  
    data_weekend,data_week = MakeData(table,time_col,time_span,expired_pat_weekend,expired_pat_week)
    possible_items = GetPossibleItems(table,item_col)

    data = Parallel(n_jobs=-1,max_nbytes=1e6)(delayed(TotalItemColumn)(col,table,time_col,item_col,data_label) for col in possible_items)
    print("Total Number of Each Item")
    print(table_label)
    data = pd.DataFrame(data,columns=['Column','Mean Count Week','Mean Count Weekend','U-Stat','P-Val'])
    #data = data.dropna()
    display(data[data['P-Val']<0.05])   
    print(' ')

#Make Amount items
amount_data = set(['LABEVENTS','PRESCRIPTIONS','PROCEDUREEVENTS_MV','CHARTEVENTS','INPUTITEMS'])
#Total Item Amount Items
for table_label,time_col,item_col,amount_col,table,data_label in clinical_data_tables:
    if(table_label in amount_data): 
        data_weekend,data_week = MakeData(table,time_col,time_span,expired_pat_weekend,expired_pat_week)
        possible_items = GetPossibleItems(table,item_col)
        data = Parallel(n_jobs=-1,max_nbytes=1e6)(delayed(TotalAmountColumn)(col,table,amount_col,item_col,data_label) for col in possible_items)
        print("Total Number of Each Item")
        print(table_label)
        data = pd.DataFrame(data,columns=['Column_mean','CountWeek_mean','CountWeekend_mean','U-Stat_mean','P-Val_mean',
                                          'Column_max','CountWeek_max','CountWeekend_max','U-Stat_max','P-Val_max',
                                          'Column_min','CountWeek_min','CountWeekend_min','U-Stat_min','P-Val_min',
                                          'Column_std','CountWeek_std','CountWeekend_std','U-Stat_std','P-Val_std'])
        data = data.dropna()
        display(data[(data['P-Val_mean']<0.05)|(data['P-Val_max']<0.05)|(data['P-Val_min']<0.05)|(data['P-Val_std']<0.05)])  

        print(' ')

caregiver_data = set(['DATETIMEEVENTS','PROCEDUREEVENTS_MV','NOTEEVENTS','INPUTITEMS'])
#Caregivers there is research that says it is due to a decrease in specialist care
for table_label,time_col,item_col,amount_col,table,data_label in clinical_data_tables: 
    if table_label in caregiver_data:
        data_weekend,data_week = MakeData(table,time_col,time_span,expired_pat_weekend,expired_pat_week)

        a = data_weekend.groupby('HADM_ID')['CGID'].unique().apply(lambda x: len([i for i in x if pd.notnull(i)])).values
        b = data_week.groupby('HADM_ID')['CGID'].unique().apply(lambda x: len([i for i in x if pd.notnull(i)])).values
        t = stats.mannwhitneyu(a,b)

        c = data_weekend.groupby('HADM_ID').size()
        d = data_week.groupby('HADM_ID').size()
        t1 = stats.mannwhitneyu(c,d)
        columns = ['Table','U-Stat-Unique','P-Value','Mean Weekend','Mean Week','U-Stat-Total','P-Value','Mean Weekend','Mean Week']
        data = pd.DataFrame([[table_label,t[0],t[1],np.mean(a),np.mean(b),t1[0],t1[1],np.mean(c),np.mean(d)]],columns = columns)
        display(data)   
        print(' ')

#Nursing Notes Breakdown
nursenotes_data = set(['NOTEEVENTS'])
for table_label,time_col,item_col,amount_col,table,data_label in clinical_data_tables: 
    if table_label in nursenotes_data:
        data_weekend,data_week = MakeData(table,time_col,time_span,expired_pat_weekend,expired_pat_week)

        for category in [['ECG'],['Nursing/other','Nursing'],['Radiology'],['Echo'],['Discharge summary'],['Physician '],['Nutrition'],['General'],
               ['Case Management '],['Respiratory '],['Rehab Services'],['Social Work'],['Pharmacy'],['Consult']]:

            data_weekend,data_week = MakeData(table,time_col,time_span,expired_pat_weekend,expired_pat_week)

            data_weekend = data_weekend[data_weekend['CATEGORY'].isin(category)]
            data_week = data_week[data_week['CATEGORY'].isin(category)]

            a = data_weekend.groupby('HADM_ID')['CGID'].unique().apply(lambda x: len([i for i in x if pd.notnull(i)])).values
            b = data_week.groupby('HADM_ID')['CGID'].unique().apply(lambda x: len([i for i in x if pd.notnull(i)])).values
            try:
                t = stats.mannwhitneyu(a,b)
            except:
                t = (np.nan,np.nan)

            c = data_weekend.groupby('HADM_ID').size()
            d = data_week.groupby('HADM_ID').size()
            try:
                t1 = stats.mannwhitneyu(c,d)
            except:
                t1 = (np.nan,np.nan)


            columns = ['Category','U-Stat-Unique','P-Value','Mean Weekend','Mean Week','U-Stat-Total','P-Value','Mean Weekend','Mean Week']
            data = pd.DataFrame([[category,t[0],t[1],np.mean(a),np.mean(b),t1[0],t1[1],np.mean(c),np.mean(d)]],columns=columns)
            display(data)   


from sklearn.cross_validation import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report
from sklearn.metrics import f1_score,recall_score,precision_score

for table_label,time_col,item_col,amount_col,table,data_label in clinical_data_tables: 
    data_weekend,data_week = MakeData(table,time_col,time_span,expired_pat_weekend,expired_pat_week)
    vector_items = MakeItemVectors(table,item_col,data_weekend,data_week)
    weekend_vectors = pd.DataFrame(vector_items[0],columns=vector_items[2])
    week_vectors = pd.DataFrame(vector_items[1],columns=vector_items[2])
    
    class_weekend = list([1]*weekend_vectors.shape[0])
    class_week = list([0]*week_vectors.shape[0])
    classes = class_weekend+class_week

    print(table_label)
    y = pd.Series(copy.deepcopy(classes))
    X = copy.deepcopy(pd.concat([weekend_vectors,week_vectors]))
    #X['intercept'] = 1.0

    sss = StratifiedKFold(y,3)

    scores = {}
    scores['F1'] = []
    scores['P'] = []
    scores['R'] = []
    for train_index, test_index in sss:
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        result = RandomForestClassifier(n_estimators=1000,random_state=33,class_weight='balanced').fit(X_train,y_train)
        y_pred = result.predict(X_test)
        scores['P'].append(precision_score(y_test,y_pred))
        scores['R'].append(recall_score(y_test,y_pred))
        scores['F1'].append(f1_score(y_test,y_pred))
        print(classification_report(y_test, y_pred))

importances = result.feature_importances_
columns = X.columns
indices = np.argsort(importances)[::-1]
for f in range(X.shape[1]):
    print("%d. feature %d (%f)" % (f + 1, columns[indices[f]], importances[indices[f]]))

temp = pd.read_csv('../MIMICIII/PRESCRIPTIONS.csv')
temp.head()

import networkx as nx
from networkx.algorithms import bipartite
B_weekend = nx.Graph()
B_weekend.add_nodes_from(data_weekend['HADM_ID'].values,bipartite=0)
B_weekend.add_nodes_from(proc_table['ITEMID'].values,bipartite=1)

for row in data_weekend.iterrows():
    subset = proc_table[proc_table['HADM_ID']==row[1]['HADM_ID']]
    for row1 in subset.iterrows():
        B_weekend.add_edge(row[1]['HADM_ID'],row1[1]['ITEMID'])
        
B_week = nx.Graph()
B_week.add_nodes_from(data_week['HADM_ID'].values,bipartite=0)
B_week.add_nodes_from(proc_table['ITEMID'].values,bipartite=1)

for row in data_week.iterrows():
    subset = proc_table[proc_table['HADM_ID']==row[1]['HADM_ID']]
    for row1 in subset.iterrows():
        B_week.add_edge(row[1]['HADM_ID'],row1[1]['ITEMID'])
        
X, Y = bipartite.sets(B_weekend)
pos = dict()
pos.update( (n, (1, i)) for i, n in enumerate(X) ) # put nodes from X at x=1
pos.update( (n, (2, i)) for i, n in enumerate(Y) ) # put nodes from Y at x=2
nx.draw(B_weekend, pos=pos)
plt.show()